In [20]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import time
from pandas import ExcelWriter
import re
import pyodbc
import asyncio
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import pdb
#pdb.set_trace()
from ipywidgets import IntProgress
from IPython.display import display
import os
from selenium.webdriver.support import expected_conditions
from IPython.display import clear_output
from datetime import timedelta

In [21]:
TIEMPO_DE_ESPERA = 4

start_ = datetime.datetime.now()
fecha = datetime.date.today()
hoy = fecha.strftime('%Y/%m/%d')



In [22]:
def createDriver():
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument(f'user-agent={user_agent}')
    chrome_options.add_argument("--window-size=1625x2444")
    # options.headless = True
    return webdriver.Chrome(options = chrome_options)

In [23]:
#fecha = fecha - timedelta(days=2)

In [ ]:

dfLinks = pd.read_excel(f'dfLinks_{fecha}.xlsx')

In [ ]:
len(dfLinks[dfLinks['sexo']=='Men'])

18

In [ ]:
men = dfLinks.loc[dfLinks['sexo'] == 'Men'].reset_index()

In [ ]:
men

,index,url,tipo,sexo
0,22,https://www2.hm.com/en_us/men/products/hoodies...,Hoodies & Sweatshirts,Men
1,23,https://www2.hm.com/en_us/men/products/shirts....,Shirts,Men
2,24,https://www2.hm.com/en_us/men/products/jackets...,Jackets & Coats,Men
3,25,https://www2.hm.com/en_us/men/products/jeans.html,Jeans,Men
4,26,https://www2.hm.com/en_us/men/products/pants.html,Pants,Men
5,27,https://www2.hm.com/en_us/men/products/cardiga...,Cardigans & Sweaters,Men
6,28,https://www2.hm.com/en_us/men/products/t-shirt...,T-shirts & Tanks,Men
7,29,https://www2.hm.com/en_us/men/products/basics....,Basics,Men
8,30,https://www2.hm.com/en_us/men/products/suits-b...,Blazers & Suits,Men
9,31,https://www2.hm.com/en_us/men/products/shoes.html,Shoes,Men


In [ ]:
nocargaron = []
lista_productos = []

browser = createDriver()
for index, row in men.iterrows():
    
    #requests de la url //ropa mujer//hombre//kids
    try:
        browser.get(row['url'])
    except:
        browser.quit()
        browser = createDriver()
        time.sleep(1)
        continue
    
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:

        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        try:
            #boton para cargar más items
            browser.find_element_by_class_name('load-more-products').find_element_by_tag_name('button').click()
        except:
            pass
        time.sleep(TIEMPO_DE_ESPERA)
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    time.sleep(1)

    CANTIDAD_PRODUCTOS = browser.execute_script('''
    var x = document.getElementsByClassName('product-item').length;
    return x;''')
    style = {'description_width': 'initial'}
    test = IntProgress(style=style, min = 0, max = CANTIDAD_PRODUCTOS, description=row['tipo']+','+row['sexo'], bar_style='danger',)    
    display(test)
    #CANTIDAD DE PRODUCTOS
    #_ no lo uso porque deja de estar en el DOM ya que hago click y se pierde el elemento, solo necesito el indice,
    for index_producto in range(CANTIDAD_PRODUCTOS):
        
        
        pos = index_producto+1
        try:
            time.sleep(3)
            browser.execute_script(f'''
            document.getElementsByClassName('hm-product-item')[{index_producto}].getElementsByTagName('a')[0].click();
            ''')
        except:
            continue

        try:
            #WebDriverWait(browser, 100).until(EC.presence_of_element_located((By.CLASS_NAME,'group')))
            WebDriverWait(browser, 90).until(EC.presence_of_element_located((By.CLASS_NAME,'mini-slider')))
        except:
            browser.get(browser.current_url)
            time.sleep(5)

        try:

            cantidad_colores = browser.execute_script('''
                            var cant = document.getElementsByClassName('group')[0].getElementsByClassName('list-item').length;
                            return cant;
                            ''')
        except:
            nocargaron.append([browser.current_url])
            browser.execute_script('history.back();')
            continue




        for color in range(cantidad_colores):


            #click en el color.
            browser.execute_script(f'''
                    document.getElementsByClassName('group')[0].getElementsByClassName('list-item')[{color}].getElementsByTagName('a')[0].click();
                    ''')


            nombre_color = browser.execute_script(f'''
                            var nombre = document.getElementsByClassName('group')[0].getElementsByClassName('list-item')[{color}].getElementsByTagName('a')[0].getAttribute('title');
                            return nombre;
                            ''')

            id_producto = browser.execute_script(f'''
                            var id = document.getElementsByClassName('group')[0].getElementsByClassName('list-item')[{color}].getElementsByTagName('a')[0].getAttribute('data-articlecode');
                            return id;
                            ''')

            descripcion = browser.execute_script('''
                            var desc = document.getElementsByClassName('primary product-item-headline')[0].textContent.trim();
                            return desc;
                            ''')

            precio = browser.execute_script('''
                        var precio = document.getElementsByClassName('price parbase')[0].textContent.trim();
                        return precio;
                        ''')

            url_producto = browser.current_url

            img = browser.execute_script('''
                    var img = document.getElementsByClassName('product-detail-main-image-container')[0].getElementsByTagName('img')[0].getAttribute('src');
                    return img;
                    ''')

            lista_productos.append([pos,
                                    id_producto,
                                    descripcion,
                                    nombre_color,
                                    row['tipo'],
                                    precio,
                                    row['sexo'],
                                    url_producto,
                                    img,
                                    row['url']])

        browser.execute_script('history.back();')
        test.value +=1
        test.description = f"± {int(CANTIDAD_PRODUCTOS)*15}: {test.value} / {CANTIDAD_PRODUCTOS}"

    test.bar_style='success'
    clear_output() 

end_ = datetime.datetime.now()


IntProgress(value=0, bar_style='danger', description='T-shirts & Tanks,Men', max=149, style=ProgressStyle(desc…

In [ ]:
index

In [31]:
men

,index,url,tipo,sexo
0,22,https://www2.hm.com/en_us/men/products/hoodies...,Hoodies & Sweatshirts,Men
1,23,https://www2.hm.com/en_us/men/products/shirts....,Shirts,Men
2,24,https://www2.hm.com/en_us/men/products/jackets...,Jackets & Coats,Men
3,25,https://www2.hm.com/en_us/men/products/jeans.html,Jeans,Men
4,26,https://www2.hm.com/en_us/men/products/pants.html,Pants,Men
5,27,https://www2.hm.com/en_us/men/products/cardiga...,Cardigans & Sweaters,Men
6,28,https://www2.hm.com/en_us/men/products/t-shirt...,T-shirts & Tanks,Men
7,29,https://www2.hm.com/en_us/men/products/basics....,Basics,Men
8,30,https://www2.hm.com/en_us/men/products/suits-b...,Blazers & Suits,Men
9,31,https://www2.hm.com/en_us/men/products/shoes.html,Shoes,Men


In [33]:
len(lista_productos)

8119

In [34]:
browser.quit()

In [35]:
df = pd.DataFrame(lista_productos)

In [36]:
df.head()

,0,1,2,3,4,5,6,7,8,9
0,1,0685814001,Hoodie,Black,Hoodies & Sweatshirts,$24.99,Men,https://www2.hm.com/en_us/productpage.06858140...,//lp2.hm.com/hmgoepprod?set=source[/bd/02/bd02...,https://www2.hm.com/en_us/men/products/hoodies...
1,1,0685814002,Hoodie,Light gray melange,Hoodies & Sweatshirts,$24.99,Men,https://www2.hm.com/en_us/productpage.06858140...,//lp2.hm.com/hmgoepprod?set=source[/d5/8a/d58a...,https://www2.hm.com/en_us/men/products/hoodies...
2,1,0685814003,Hoodie,White,Hoodies & Sweatshirts,$24.99,Men,https://www2.hm.com/en_us/productpage.06858140...,//lp2.hm.com/hmgoepprod?set=source[/a2/7d/a27d...,https://www2.hm.com/en_us/men/products/hoodies...
3,1,0685814005,Hoodie,Beige,Hoodies & Sweatshirts,$12.99$24.99,Men,https://www2.hm.com/en_us/productpage.06858140...,//lp2.hm.com/hmgoepprod?set=source[/e7/90/e790...,https://www2.hm.com/en_us/men/products/hoodies...
4,1,0685814006,Hoodie,Burgundy,Hoodies & Sweatshirts,$24.99,Men,https://www2.hm.com/en_us/productpage.06858140...,//lp2.hm.com/hmgoepprod?set=source[/94/e9/94e9...,https://www2.hm.com/en_us/men/products/hoodies...


In [37]:
df.rename(columns={0:'pos',
                  1:'sku',
                  2:'descripcion',
                  3:'color',
                  4:'tipo',
                  5:'precio',
                  6:'sexo',
                  7:'url',
                  8:'jpg',
                  9:'pagina_scraper'},inplace=True)

In [38]:
df.head()

,pos,sku,descripcion,color,tipo,precio,sexo,url,jpg,pagina_scraper
0,1,0685814001,Hoodie,Black,Hoodies & Sweatshirts,$24.99,Men,https://www2.hm.com/en_us/productpage.06858140...,//lp2.hm.com/hmgoepprod?set=source[/bd/02/bd02...,https://www2.hm.com/en_us/men/products/hoodies...
1,1,0685814002,Hoodie,Light gray melange,Hoodies & Sweatshirts,$24.99,Men,https://www2.hm.com/en_us/productpage.06858140...,//lp2.hm.com/hmgoepprod?set=source[/d5/8a/d58a...,https://www2.hm.com/en_us/men/products/hoodies...
2,1,0685814003,Hoodie,White,Hoodies & Sweatshirts,$24.99,Men,https://www2.hm.com/en_us/productpage.06858140...,//lp2.hm.com/hmgoepprod?set=source[/a2/7d/a27d...,https://www2.hm.com/en_us/men/products/hoodies...
3,1,0685814005,Hoodie,Beige,Hoodies & Sweatshirts,$12.99$24.99,Men,https://www2.hm.com/en_us/productpage.06858140...,//lp2.hm.com/hmgoepprod?set=source[/e7/90/e790...,https://www2.hm.com/en_us/men/products/hoodies...
4,1,0685814006,Hoodie,Burgundy,Hoodies & Sweatshirts,$24.99,Men,https://www2.hm.com/en_us/productpage.06858140...,//lp2.hm.com/hmgoepprod?set=source[/94/e9/94e9...,https://www2.hm.com/en_us/men/products/hoodies...


In [20]:
#df['precio'] = df['precio'].apply(lambda x: float(x.split()[1]))

In [39]:
df.to_excel(f'./Salida/HyM_men{fecha}.xlsx')

In [40]:
browser.quit()